In [63]:
import datasets

# Loading an existing dataset

In [25]:
from datasets import load_dataset

dataset = load_dataset("imdb")
# Show an example data point
dataset['train'][0]


{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [26]:
# Exploring the Dataset
# How to explore the dataset structure (e.g., columns, data types).
# Show the structure
print(dataset['train'].features)

# Display first five entries
print(dataset['train'][:5])
dataset['train'] = dataset['train'].select(range(100))
dataset['train']

{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}
{'text': ['I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 yea

Dataset({
    features: ['text', 'label'],
    num_rows: 100
})

# load a custom dataset using script

In [64]:
ner_dataset = datasets.load_dataset(path="conll.py", trust_remote_code=True)
ner_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

# load a custom dataset


In [27]:
class InputExample(object):
    """A single training/test example for token classification."""

    def __init__(self, guid, words, labels):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            words: list. The words of the sequence.
            labels: (Optional) list. The labels for each word of the sequence. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.words = words
        self.labels = labels


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids


In [28]:

import os
HP_SEARCH_DATA_SIZE = 128
import random
import functools

def read_examples_from_file(data_dir, mode, batch_size, is_hyper_parameter_search = False):
    total_examples = []
    breakpoint()
    examples = []
    file_path = data_dir
    print(file_path)
    guid_index = 1
    
    with open(file_path, encoding="utf-8") as f:
        words = []
        labels = []
        for line in f:
            if line.startswith("-DOCSTART-") or line == "" or line == "\n":
                if words:
                    examples.append(InputExample(guid="{}-{}".format(mode, guid_index),
                                                words=words,
                                                labels=labels))
                    guid_index += 1
                    words = []
                    labels = []
            else:
                splits = line.split(" ")
                words.append(splits[0].lower())
                if len(splits) > 1:
                    labels.append(splits[-1].replace("\n", "").strip())
                else:
                    # Examples could have no label for mode = "test"
                    labels.append("O")
            if guid_index > 4 * HP_SEARCH_DATA_SIZE:
                break
        if words:
            examples.append(InputExample(guid="%s-%d".format(mode, guid_index),
                                        words=words,
                                        labels=labels))
            

        total_examples.append(examples)
    min_len = min([len(ex) for ex in total_examples])
    [random.shuffle(ex) for ex in total_examples]
    total_examples = [ex[:min_len] for ex in total_examples]
    total_examples = functools.reduce(lambda a, b: a+b, total_examples)
    extra_example_size = (len(total_examples) % batch_size)
    print("Number of examples: {}".format(len(total_examples[:-(len(total_examples) % batch_size)])))
    random.shuffle(total_examples)
    if is_hyper_parameter_search:
        total_examples = total_examples[:HP_SEARCH_DATA_SIZE]
        return total_examples
    return total_examples if extra_example_size == 0 else total_examples[:-(len(total_examples) % batch_size)]


def convert_examples_to_features(examples,
                                 label_list,
                                 max_seq_length,
                                 tokenizer,
                                 cls_token_at_end=False,
                                 cls_token="[CLS]",
                                 cls_token_segment_id=1,
                                 sep_token="[SEP]",
                                 sep_token_extra=False,
                                 pad_on_left=False,
                                 pad_token=0,
                                 pad_token_segment_id=0,
                                 pad_token_label_id=-1,
                                 sequence_a_segment_id=0,
                                 mask_padding_with_zero=True
                                ):
    """ Loads a data file into a list of `InputBatch`s
        `cls_token_at_end` define the location of the CLS token:
            - False (Default, BERT/XLM pattern): [CLS] + A + [SEP] + B + [SEP]
            - True (XLNet/GPT pattern): A + [SEP] + B + [SEP] + [CLS]
        `cls_token_segment_id` define the segment id associated to the CLS token (0 for BERT, 2 for XLNet)
    """

    label_map = {label: i for i, label in enumerate(label_list)}
    print('label map', label_map)

    features = []
    # examples = examples[:10000]
    for (ex_index, example) in enumerate(examples):
        #if ex_index % 10000 == 0:
        #    print("Writing example %d of %d", ex_index, len(examples))

        tokens = []
        label_ids = []
        i = 0
        for word, label in zip(example.words, example.labels):
            if label == '' or label == ' ' or word == '' or word == ' ':
                continue
            i += 1
            word_tokens = tokenizer.tokenize(word)
            tokens.extend(word_tokens)
            # print(word)
            # Use the real label id for the first token of the word, and padding ids for the remaining tokens
            label_ids.extend([label_map[label]] + [pad_token_label_id] * (len(word_tokens) - 1))

        # Account for [CLS] and [SEP] with "- 2" and with "- 3" for RoBERTa.
        special_tokens_count = 3 if sep_token_extra else 2
        if len(tokens) > max_seq_length - special_tokens_count:
            tokens = tokens[:(max_seq_length - special_tokens_count)]
            label_ids = label_ids[:(max_seq_length - special_tokens_count)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids:   0   0  0    0    0     0       0   0   1  1  1  1   1   1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids:   0   0   0   0  0     0   0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens += [sep_token]
        label_ids += [pad_token_label_id]
        if sep_token_extra:
            # roberta uses an extra separator b/w pairs of sentences
            tokens += [sep_token]
            label_ids += [pad_token_label_id]
        segment_ids = [sequence_a_segment_id] * len(tokens)

        if cls_token_at_end:
            tokens += [cls_token]
            label_ids += [pad_token_label_id]
            segment_ids += [cls_token_segment_id]
        else:
            tokens = [cls_token] + tokens
            label_ids = [pad_token_label_id] + label_ids
            segment_ids = [cls_token_segment_id] + segment_ids

        input_ids = tokenizer.convert_tokens_to_ids(tokens)


        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding_length = max_seq_length - len(input_ids)
        padding_length_label = max_seq_length - len(label_ids)
        # print(padding_length, padding_length_label)
        if pad_on_left:
            input_ids = ([pad_token] * padding_length) + input_ids
            input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
            segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
            label_ids = ([pad_token_label_id] * padding_length_label) + label_ids
        else:
            input_ids += ([pad_token] * padding_length)
            input_mask += ([0 if mask_padding_with_zero else 1] * padding_length)
            segment_ids += ([pad_token_segment_id] * padding_length)
            # print(len(label_ids))
            label_ids += ([pad_token_label_id] * padding_length_label)
            # print(len(label_ids))

        # print('\n\n\nMax seq len:', max_seq_length, 'Label ids len',  len(label_ids))
        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        assert len(label_ids) == max_seq_length

        if ex_index < 5:
           print("*** Example ***")
           print("guid: %s", example.guid)
           print("tokens: %s", " ".join([str(x) for x in tokens]))
           print("input_ids: %s", " ".join([str(x) for x in input_ids]))
           print("input_mask: %s", " ".join([str(x) for x in input_mask]))
           print("segment_ids: %s", " ".join([str(x) for x in segment_ids]))
           print("label_ids: %s", " ".join([str(x) for x in label_ids]))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_ids=label_ids))
    return features


In [29]:
example_path = "../data/conll_data.txt"

examples = read_examples_from_file(
    data_dir=example_path, 
    mode="train",
    batch_size=16
    )

../data/conll_data.txt
Number of examples: 336


In [30]:
from transformers import AutoTokenizer
from torch.nn import CrossEntropyLoss
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
pad_token_label_id = CrossEntropyLoss().ignore_index
labels = ['O','I-AGG','I-ATR','I-ENT','I-FLT','I-FLO','I-NUM','I-PRW']
features = convert_examples_to_features(examples, 
                                        labels, 
                                        16, 
                                        tokenizer,
                                        )

label map {'O': 0, 'I-AGG': 1, 'I-ATR': 2, 'I-ENT': 3, 'I-FLT': 4, 'I-FLO': 5, 'I-NUM': 6, 'I-PRW': 7}
*** Example ***
guid: %s train-2
tokens: %s [CLS] predict the average airline delay for each airline tomorrow [SEP]
input_ids: %s 101 17163 1103 1903 8694 8513 1111 1296 8694 4911 102 0 0 0 0 0
input_mask: %s 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0
segment_ids: %s 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
label_ids: %s -1 0 0 1 2 2 0 0 3 7 -1 -1 -1 -1 -1 -1
*** Example ***
guid: %s train-34
tokens: %s [CLS] predict the average air system delay for flights for each destination airport which will [SEP]
input_ids: %s 101 17163 1103 1903 1586 1449 8513 1111 7306 1111 1296 7680 3871 1134 1209 102
input_mask: %s 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
segment_ids: %s 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
label_ids: %s -1 0 0 1 2 2 2 0 0 0 0 3 3 0 0 -1
*** Example ***
guid: %s train-332
tokens: %s [CLS] predict the average change is duration of flights for flights which will land in [SEP]
input_ids: %s 101 17163 1103 1903 

/home/sibat/miniconda3/envs/hfess/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [56]:
import torch
from torch.utils.data import TensorDataset
all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_ids for f in features], dtype=torch.long)

dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
dataset

# working with csv with datasets

### Example: Loading a List as a Dataset
Suppose you have a list of dictionaries, where each dictionary represents a data point in your dataset. 

In [33]:
from datasets import Dataset
data = [
    {'id': 1, 'text': 'Hello world', 'label': 1},
    {'id': 2, 'text': 'HuggingFace is amazing', 'label': 0},
    {'id': 3, 'text': 'Machine learning is fun', 'label': 1}
]

# Create a Dataset from a list of dictionaries
dataset = Dataset.from_list(data)

dataset


Dataset({
    features: ['id', 'text', 'label'],
    num_rows: 3
})

### Example: Loading a Dictionary as a Dataset
Suppose you have a dictionary where the keys represent different features (e.g., id, text, label) and the values are lists containing the data for each feature.

In [34]:
data = {
    'id': [1, 2, 3],
    'text': ['Hello world', 'HuggingFace is amazing', 'Machine learning is fun'],
    'label': [1, 0, 1]
}
dataset = Dataset.from_dict(data)
dataset

Dataset({
    features: ['id', 'text', 'label'],
    num_rows: 3
})

### Example: Loading a JSON File as a Dataset
First, ensure that your JSON file is structured properly. For example, the JSON might look something like this if saved in a file named data.json:

In [37]:
json_data = Dataset.from_json("../data/dummy_json.json")
json_data


Generating train split: 3 examples [00:00, 399.77 examples/s]


Dataset({
    features: ['id', 'text', 'label'],
    num_rows: 3
})

### Example: Loading a CSV File as a Dataset


In [61]:
data = Dataset.from_csv("../data/dummy_csv.csv")
data


Dataset({
    features: ['id', 'text', 'label'],
    num_rows: 3
})

In [40]:
# Removing columns as explained previously
data = data.remove_columns(['id'])
data

Dataset({
    features: ['text', 'label'],
    num_rows: 3
})

### Common functions

In [62]:
def lowercase_text(example):
    example['text'] = example['text'].lower()
    return example

def uppercase_text(example):
    example['text'] = example['text'].upper()
    return example

data = data.map(lowercase_text)

print(data['text'])
data = data.map(uppercase_text)
print("Uppercase Text: " + str(data['text']))

def add_a_dummy_column(item):
    item.update({'dummy': 'dummy'})
    return item

data = data.map(add_a_dummy_column)
print(f"After adding a new column: {data}")

print(data.filter(lambda x: 'WORLD' in x['text']))

data = data.shuffle(seed=42)
print(f"After Shuffling: {data['text']}")
    

Map: 100%|██████████| 3/3 [00:00<00:00, 793.12 examples/s]


['hello world', 'huggingface is amazing', 'machine learning is fun']


Map: 100%|██████████| 3/3 [00:00<00:00, 915.19 examples/s]


Uppercase Text: ['HELLO WORLD', 'HUGGINGFACE IS AMAZING', 'MACHINE LEARNING IS FUN']


Map: 100%|██████████| 3/3 [00:00<00:00, 864.86 examples/s]


After adding a new column: Dataset({
    features: ['id', 'text', 'label', 'dummy'],
    num_rows: 3
})


Filter: 100%|██████████| 3/3 [00:00<00:00, 1102.31 examples/s]

Dataset({
    features: ['id', 'text', 'label', 'dummy'],
    num_rows: 1
})
After Shuffling: ['MACHINE LEARNING IS FUN', 'HUGGINGFACE IS AMAZING', 'HELLO WORLD']


In [59]:
data = Dataset.from_csv("../data/small_data.csv")
train_test_split = data.train_test_split(test_size=0.2)
train_test_split

Generating train split: 15 examples [00:00, 4736.83 examples/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label'],
        num_rows: 12
    })
    test: Dataset({
        features: ['id', 'text', 'label'],
        num_rows: 3
    })
})